In [ ]:
import json
import pandas as pd
from datetime import date, datetime
from dotenv import dotenv_values
from git import Repo, GitCommandError
from pathlib import Path
import dapla as dp

# Read in magic values
config = dotenv_values("../../.env")
EXCLUDE_REPOS = json.loads(config["EXCLUDE_REPOS"])
PATH_TO_CONFIG_FILE = config["PATH_TO_CONFIG_FILE"]
FIRST_COMMIT_SHA = config["FIRST_COMMIT_SHA"]
CSV_FILE_NAME = config["CSV_FILE_NAME"]

TEAMS_COLUMN_NAME = "number_of_teams"

# Initialise DataFrame
output_data = pd.DataFrame(columns=["date", TEAMS_COLUMN_NAME]).set_index("date")

In [ ]:
def get_number_of_teams_at_ref(git_ref: str) -> int:
    # Get contents of file for this commit
    file_contents = atlantis_config.git.show('{}:{}'.format(git_ref, PATH_TO_CONFIG_FILE))
    line = [line.strip() for line in file_contents.splitlines() if line.strip().startswith("value: github.com")][0]
    sections = [l for l in line.split(",") if not(any([r in l for r in EXCLUDE_REPOS]))]
    return len(sections)

In [ ]:
# Assumes the atlantis-team-config repo is cloned to the home directory
atlantis_config = Repo(Path.home() / "atlantis-team-config")

i = 0
for commit in atlantis_config.iter_commits("main", max_count=200):
    if commit.hexsha.startswith(FIRST_COMMIT_SHA):
        break
    try:
        # Record the datapoint in the Dataframe
        output_data.at[commit.committed_datetime.date(), TEAMS_COLUMN_NAME] = get_number_of_teams_at_ref(commit.hexsha)
    except GitCommandError:
        break

# Special case: Always record the latest datapoint
output_data.at[date.today(), TEAMS_COLUMN_NAME] = get_number_of_teams_at_ref("HEAD")
print(output_data)

In [ ]:
import plotly
import plotly.io as pio
import plotly.express as px
import nbformat

pd.options.plotting.backend = "plotly"

output_data.sort_index(inplace=True)
output_data = output_data.truncate(before=date(2022, 3, 15), after=datetime.now().date())
figure = px.line(
    output_data,
    title="Team etablert på Dapla",
    template="plotly_dark",
    labels=dict(index="Dato", value="Antall team", variable="Antall")
)
figure.update_layout(showlegend=False)
figure.show()

In [ ]:
dp.FileClient.save_pandas_to_csv(output_data, CSV_FILE_NAME, index=True)
print(dp.FileClient.cat(CSV_FILE_NAME))